# Experiment description
## Hypothesis: 
We can train sepp model at localidad aggregation using three-weeks data

## Method: 

Train model using three-weeks data (2017-01-01 to 2017-01-22) at localidad scope. Use parameters configuration successful on experiment 04 (one week training).

## Results:


In [ ]:
%matplotlib inline
import pandas as pd
import pickle
import dateutil.parser
import pyproj
import open_cp
import open_cp.sepp as sepp
from PIL import Image
import datetime
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
import import_ipynb
import training_sepp_builtin

In [ ]:
def selectDataTraining(df,nameLoc,dateIni,dateEnd):
    if nameLoc == 'city':
        dfloc = df
    else:
        dfloc = df[df["LOCALIDAD"]==nameLoc]
        
    dfloc["FECHA_HORA"]=dfloc["HORA"].astype(str)
    valHour = dfloc["FECHA_HORA"].values
    valDate = dfloc["FECHA"].values
    timesVals = [];
    k = 0;
    for i in valHour:
        if len(i)<=2:
            timeStr = valDate[k] +" " + "00:"+i+":00"
        else:
            timeStr = valDate[k] +" " + i[:-2]+":"+i[-2:]+":00"
        
        k = k + 1

        timesVals.append(timeStr)
    dfloc["FECHA_HORA"] = timesVals;
    
    dfloc["FECHA_HORA_TS"] = dfloc.FECHA_HORA.map(dateutil.parser.parse)
    dfFilter = dfloc[(dfloc.FECHA_HORA_TS > dateIni) & (dfloc.FECHA_HORA_TS < dateEnd)]
    timestamps = dfFilter.FECHA_HORA_TS
    
    print("TimeStamps")
    print(type(timestamps))

    xcoords, ycoords = (dfFilter.LONGITUD.values,dfFilter.LATITUD.values)
    proj = pyproj.Proj(init="EPSG:3116")
    xcoords, ycoords = proj(xcoords,ycoords)
    
    points_train = open_cp.TimedPoints.from_coords(timestamps, xcoords, ycoords)
    return (points_train,timestamps,xcoords, ycoords)

In [ ]:
#default parameters
parameters = {"time_bw":144,"space_bw":50,"time_cutoff":172800,"space_cutoff":500}

def trainModel_parameters(df,nameLoc,dateIni,dateEnd,parameters,iterations=10):
    points_train,timestamps,xcoords, ycoords = selectDataTraining(df,nameLoc,dateIni,dateEnd)
    plt.scatter(xcoords, ycoords)
    print([timestamps])
    trainer = sepp.SEPPTrainer()
    trainer._initial_time_bandwidth = parameters['time_bw']
    trainer._initial_space_bandwidth = parameters['space_bw']
    trainer._time_cutoff = parameters['time_cutoff']
    trainer._space_cutoff = parameters['space_cutoff']
    print('time BW set on: ',trainer._initial_time_bandwidth)
    print('space BW set on: ',trainer._initial_space_bandwidth)
    print('time cutoff set on: ',trainer._time_cutoff)
    print('space cutoff set on: ',trainer._space_cutoff)
    print('Number of realizations: ',iterations)
    trainer.data = points_train    
    while True:  
        try:
            predictor = trainer.train()
            print("succesful training!")
            break
        except (RuntimeError, ValueError) as e:
            print(e)
            pass
        iterations = iterations - 1
        if iterations == 0:
            return "not succesfull training"
    return predictor 

## Select data

In [ ]:
path = '/Users/anamaria/Desktop/dev/security_project'

In [ ]:
df = pd.read_csv(path+'/datasets/verify_enrich_nuse_29112019.csv')

In [ ]:
initial_date = '2017-01-01'
final_date = '2017-01-22'

In [ ]:
df.LOCALIDAD.unique()

## Training experiments

### Scenario 1: successful parameters used on experiment 04
time_cutoff = [30, 50, 70, 90, 120]

In [ ]:
#default parameters
parameters = {"time_bw":144,"space_bw":50,"time_cutoff":172800,"space_cutoff":500}

In [ ]:
localidad = 'PUENTE ARANDA'
parameters['time_cutoff'] = 120
iterations = 100
city_predictor = trainModel_parameters(df,localidad,initial_date, final_date,parameters,iterations)
if city_predictor != "not succesfull training":    
    experiment_name = 'localidad_'+str(localidad)+'_'+initial_date+'_'+final_date+'_time_cutoff_'+str(parameters['time_cutoff'])
    city_outfile = open(path+'/aggressive_behavior_model/SEPP/'+'pkl/TrainRina_'+experiment_name+'.pkl','wb')
    pickle.dump(city_predictor, city_outfile)
    city_outfile.close() 